# Mount Drive if Necessary

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/EPFL/25spring/CS439')

# Imports and Prepare Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import STL10
from torchvision import transforms
from torch.utils.data import DataLoader, ConcatDataset, random_split, Subset
from torch.nn.functional import interpolate
from torchvision.models import resnet50, ResNet50_Weights
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
from zo_utils import*
from zo_attacks import*


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_path = "resnet50_stl10.pth" #change if necessary


# Reinitialize model structure
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
model.fc = nn.Linear(2048, 10)

# Load trained weights
model.load_state_dict(torch.load("models/resnet50_stl10.pth"))

# Move to GPU and eval mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# Experiments

In [ ]:
# Get Data

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(96, padding=12),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = STL10(root='./data', split='train', download=True, transform=transform)
testset = STL10(root='./data', split='test', download=True, transform=transform)


trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=0)
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=0)



In [ ]:
image, label = testloader.dataset[4]
image = image.unsqueeze(0).to(device)
label = torch.tensor([label], device=device)


In [ ]:
# Zero order PGD

adv, pred = zero_pgd_attack(model, image, label.item(), steps=200, num_dirs=100)
plot_original_vs_adversarial(image, adv, label.item(), pred, title="Zero order PGD")


In [ ]:
# Zero order projected ADAM

adv, pred = zero_pgd_attack_adam(model, image, label.item(), steps=200, num_dirs=100)
plot_original_vs_adversarial(image, adv, label.item(), pred, title="Zero order projected ADAM")


In [ ]:
# ZOO with GD

adv, pred = zoo_attack(model, image, label.item(), steps=200, num_dirs=100, alpha=100, lr=0.2)
plot_original_vs_adversarial(image, adv, label.item(), pred, title="ZOO with GD")


In [ ]:
# ZOO with ADAM

adv, pred = zoo_attack_adam(model, image, label.item(), steps=200, num_dirs=100, lr=0.2, alpha=100)
plot_original_vs_adversarial(image, adv, label.item(), pred, title="ZOO with ADAM")


In [ ]:
# Bandit with L2 norm

adv, pred = bandit_l2_attack(model, image, label.item(), steps=200, lr=0.5, eta=0.1, delta=1e-2, epsilon=1e-2, num_dirs=100,)
plot_original_vs_adversarial(image, adv, label.item(), pred, title="Bandit with L2")

In [ ]:
# Bandit with L∞ norm

adv, pred = bandit_linf_attack(model, image, label.item(), steps=200, lr=1.0, eta=100, delta=1.0, epsilon=0.1, num_dirs=100)
plot_original_vs_adversarial(image, adv, label.item(), pred, title="Bandit with L∞")

In [ ]:
# Square Attack

adv, pred = square_attack(model, image, label.item(), steps=500)
plot_original_vs_adversarial(image, adv, label.item(), pred, title="Square Attack")